In [22]:
from scipy.sparse import diags
#Saurav Adhikari - 1622912
#Lalita Awasthi - 1622924
#Nila Ravindran - 1614113

import numpy as np
from scipy.sparse.linalg import cg
import time


def conjugate_grad(Q, b, x=None):
    """
    Description
    ----------
    Solve a linear-quadratic problem min 1/2 x^TQx-b^Tx with conjugate gradient method with Q spd.
    Parameters
    ----------
    Q: 2d numpy.array of positive semi-definite (symmetric) matrix
    b: 1d numpy.array
    x: 1d numpy.array of initial point
    Returns
    -------
    1d numpy.array x such that Qx = b
    """
    n = len(b)
    if not x:
        x = np.ones(n)
    r = np.dot(Q, x) - b
    p = - r
    r_k_norm = np.dot(r, r)
    for i in range(2*n):
        Qp = np.dot(Q, p) # can be replaced by a function evaluation of Q times p
        alpha = r_k_norm / np.dot(p, Qp)
        x += alpha * p
        r += alpha * Qp
        r_kplus1_norm = np.dot(r, r)
        beta = r_kplus1_norm / r_k_norm
        r_k_norm = r_kplus1_norm
        if r_kplus1_norm < 1e-8:
            print ('Itr:', i)
            break
        p = beta * p - r
    return x


def prec_conjugate_grad(Q, W, b, x=None):
    """
    Description
    ----------
    Solve a linear-quadratic problem min 1/2 x^TQx-b^Tx with preconditioned conjugate gradient method with Q spd.
    Parameters
    ----------
    Q: 2d numpy.array of positive semi-definite (symmetric) matrix
    W: 2d numpy.array preconditioning matrix
    b: 1d numpy.array
    x: 1d numpy.array of initial point
    Returns
    -------
    1d numpy.array x such that Qx = b
    """
    n = len(b)
    if not x:
        x = np.ones(n)
    r = np.dot(Q, x) - b
    z = np.dot(W, r)
    p = - z
    r_k_z = np.dot(r, z)
    for i in range(2*n):
        Qp = np.dot(Q, p)
        alpha = r_k_z / np.dot(p, Qp)
        x += alpha * p
        r += alpha * Qp
        z = np.dot(W, r)
        r_k_z_plus1 = np.dot(r, z)
        beta = r_k_z_plus1 / r_k_z
        r_k_z = r_k_z_plus1
        if np.dot(r, r) < 1e-8:
            print ('Itr:', i)
            break
        p = beta * p - z
    return x

#Task 1

#defining the array for multiple n values
n =[10,50,250,1250]

#for first question
for x in n:
    #Creating a diagonal matrix Q as ex 2 part 1 matrix
    Q=(np.diag((np.arange(x)+1)))

    #Constructing D as sqrt(inv(diag(Q)))
    D=np.linalg.inv(np.diag(np.diag(np.diag((np.arange(x)+1)))))

    #Defining matrix b as 1 given
    b = np.ones(x)

    #Start time
    t1=time.time()
    #Computing CG
    x1=conjugate_grad(Q,b)
    #End time
    t2=time.time()

    #Time difference
    print (t2 - t1)

    #Start time
    t3=time.time()

    #Preconditioned Conjugate Gradient
    x2=prec_conjugate_grad(Q,D,b)

    #end Time
    t4=time.time()
    print(t4-t3)

#for second matrix
#defining the array for multiple n values
n =[10,50,250,1250]
for i in n:

    #Constructing a tridiagonal matrix
    Q= 2* np.eye(i) - np.eye(i,k=1) - np.eye(i,k=-1)

    #Constructing D as sqrt(inv(diag(Q)))
    D=np.linalg.inv(np.diag(np.diag(Q)))

    #given vector b of ones
    b = np.ones(i)

    #start time
    t1=time.time()
    #conjugate gradient
    x1=conjugate_grad(Q,b)
    #end time
    t2=time.time()
    #time difference
    print (t2 - t1)
    t3=time.time()

    #Preconditioned conjugate gradient
    x2=prec_conjugate_grad(Q,D,b)
    t4 = time.time()
    print (t4 - t3)


Itr: 8
0.0009982585906982422
Itr: 0
0.0010042190551757812
Itr: 27
0.0009963512420654297
Itr: 0
0.0
Itr: 67
0.0039997100830078125
Itr: 0
0.0010008811950683594
Itr: 155
0.7758419513702393
Itr: 0
0.015626907348632812
Itr: 4
0.0
Itr: 4
0.0
Itr: 23
0.0
Itr: 23
0.0
Itr: 124
0.0
Itr: 124
0.0
Itr: 623
0.38579487800598145
Itr: 623
0.7850172519683838


In [23]:
from numpy.linalg import norm
from scipy.optimize import root_scalar
from cg_test import *

def nlcg(f, x, tol=1e-9, max_it = 1000, callback=None):
    n = x.size
    beta = 0
    p = np.zeros(n)
    r=grad(f,x)
    res_new  = norm(r)
    i=0
    while res_new >= tol and i < max_it:
        i=i + 1
        p = -r + beta*p

        # Solving using root.scalar
        alpha = root_scalar(f=lambda alpha: grad(f, x+alpha*p)@p, x0=0, x1=2, method="secant").root
        if not callback is None:
            callback(x)
        x = x + alpha*p
        old_r = r

        #Replacing residual by gradient
        r = grad(f,x)
        res_old = res_new
        res_new = norm(r)
        betaPR = (r@(r-old_r))/res_new
        # Replacing beta by max (Beta**pr,0)
        beta = max(0,betaPR)*(0!=i%n) #beta = res_new**2/res_old**2
        # The term *(0!=k%n) restarts the algorithm after #dim iterations.
    if not callback is None:
        callback(x)
    if i==max_it:
        print("max itt reached")
    return  x

test(nlcg)


# Test nlcg on Rosenbrock function.
Test (should be 0):
 3.798810147401224e-20


In [29]:
#ii)
#constructing the tridiagonal matrix
import scipy.sparse as scs
n=50000
Q= (2* np.eye(n) - np.eye(n,k=1) - np.eye(n,k=-1))

b=np.ones(n)
for i in b:
    if i % 2 == 0:
        b[i] = -1

x  = conjugate_grad(Q,b)
print(x)

Itr: 2498
[2500. 4999. 7497. ... 7497. 4999. 2500.]
